In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

import requests
import json

from tqdm import tqdm_notebook

import time

In [2]:
KillerCronicasDF = pd.read_csv('CodeSwitching_Text/Killer_Cronicas-output.txt', sep = ',', quoting = 3)

KillerCronicasDF.head()

,Token,Tag
0,The,Eng
1,list,Eng
2,is,Eng
3,rather,Eng
4,voluptuously,Eng


## Initial Creation

In [3]:
SentenceTerminators = ['.','!','?']

SentenceList = []
TagList = []

currentSentence = ""
currentTagList = ""

for index, row in KillerCronicasDF.iterrows():
    currentToken = row['Token']
    currentTag = row['Tag']

    if ((currentTag == 'Punct') & (currentToken in SentenceTerminators)):
        currentSentence += (" " + currentToken)
        
        SentenceList.append(currentSentence)
        TagList.append(currentTagList)

        currentSentence = ""
        currentTagList = ""
    else:
        currentSentence += (" " + currentToken)
        currentTagList += (" " + currentTag)

SentenceDF = pd.DataFrame({'Sentence': SentenceList, 'Tags' : TagList})

SentenceDF['SentenceFrequency'] = np.nan
SentenceDF['EnglishTrans'] = ""
SentenceDF['EnglishFreq'] = np.nan
SentenceDF['SpanishTrans'] = ""
SentenceDF['SpanishFreq'] = np.nan

SentenceDF.to_csv('SentenceDatabase.csv', index = False)

## Translation Creation

In [4]:
SentenceDF = pd.read_csv('SentenceDatabase.csv', dtype = {'EnglishTrans':str,'SpanishTrans':str})

parameters = { 'auth_key' : '5e299c0b-ea07-caf5-a15e-b924ed542b08',
                'text' : 'Error',
                'target_lang' : 'en',
                'source_lang' : 'es',
                'preserve_formatting' : 1,
                'split_sentences' : 0}
requestUrl = 'https://api.deepl.com/v2/translate'

In [5]:

for index, row in tqdm_notebook(SentenceDF.iterrows(), total=SentenceDF.shape[0]):
    try:

        if (pd.isna(row['EnglishTrans'])):

            parameters['text'] = str(row['Sentence'])

            response = requests.get(requestUrl, params = parameters)

            if (response.status_code == 200):

                translatedText = json.loads(response.content.decode('utf-8'))['translations'][0]['text']

                SentenceDF.at[index, 'EnglishTrans'] = translatedText

                if (index % 500 == 0):
                    SentenceDF.to_csv('SentenceDatabase.csv', index = False)

                time.sleep(0.05)
    except:
        SentenceDF.to_csv('SentenceDatabase.csv', index = False)
        raise Exception('Something broke at index ', index)

SentenceDF.to_csv('SentenceDatabase.csv', index = False)

In [6]:
parameters = { 'auth_key' : '5e299c0b-ea07-caf5-a15e-b924ed542b08',
                'text' : 'Error',
                'target_lang' : 'es',
                'source_lang' : 'en',
                'preserve_formatting' : 1,
                'split_sentences' : 0}

In [7]:

for index, row in tqdm_notebook(SentenceDF.iterrows(), total=SentenceDF.shape[0]):
    try:

        if (pd.isna(row['SpanishTrans'])):

            parameters['text'] = str(row['Sentence'])

            response = requests.get(requestUrl, params = parameters)

            if (response.status_code == 200):

                translatedText = json.loads(response.content.decode('utf-8'))['translations'][0]['text']

                SentenceDF.at[index, 'SpanishTrans'] = translatedText

                if (index % 500 == 0):
                    SentenceDF.to_csv('SentenceDatabase.csv', index = False)

                time.sleep(0.025)
    except:
        SentenceDF.to_csv('SentenceDatabase.csv', index = False)
        raise Exception('Something broke at index ', index)

SentenceDF.to_csv('SentenceDatabase.csv', index = False)

In [8]:

EnglishMissing = []
SpanishMissing = []

for index, row in tqdm_notebook(SentenceDF.iterrows(), total=SentenceDF.shape[0]):
    if (pd.isna(row['EnglishTrans'])):
        EnglishMissing.append(index)
    
    if (pd.isna(row['SpanishTrans'])):
        SpanishMissing.append(index)

print("English Translations Missing:")
print(EnglishMissing)
print("Spanish Translations Missing:")
print(SpanishMissing)



English Translations Missing:
[1, 820, 1938]
Spanish Translations Missing:
[1, 820, 1938]
